In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [3]:
from utils.intermediate_layer_extraction import return_layer_input
from utils.ri_calculator import computeRi,computeRiNaif
import deepmatcher as dm
import torch
from tqdm import tqdm
import pandas as pd

In [6]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.load_state('../../models/walmartamazon_hybrid.pth')

In [8]:
negatives,neg_ids = return_layer_input('../../Structured/Walmart-Amazon/','test_negatives',128,
                                      hybrid_model,hybrid_model.classifier,device=-1)

In [9]:
positives,pos_ids = return_layer_input('../../Structured/Walmart-Amazon','test_positives',128,
                                      hybrid_model,hybrid_model.classifier,device=-1)

In [10]:
hybrid_model = hybrid_model.cuda()

## Compare naif approach with gradient approach

### Negatives

In [11]:
def compareMethods(df1,df2,columns):
    best1,best2 = (0,0)
    for i in df1.index:
        for col in columns:
            lval = df1.at[i,col]
            rval = df2.at[i,col]
            if(lval == 0 and rval >0) or(rval <lval and rval >0):
                best2+=1
            elif(rval ==0 and lval >0)or(lval < rval and lval >0):
                best1+=1
    return (best1,best2)

In [13]:
attributes = ['title','category','brand','modelno','price']
attribute_len = int(len(negatives[0][0])/len(attributes))

In [14]:
neg_ri,neg_ri_norms = computeRi(hybrid_model.classifier,attributes,negatives,attribute_len,1)

100%|██████████| 128/128 [00:14<00:00,  9.08it/s]


In [15]:
neg_ri_naif,neg_ri_norms_naif = computeRiNaif(negatives,positives,1,hybrid_model.classifier,attributes,attribute_len)

100%|██████████| 128/128 [00:09<00:00, 14.03it/s]


In [16]:
neg_variation_gradient = pd.DataFrame(data= neg_ri_norms,columns=attributes)
neg_variation_naif = pd.DataFrame(data= neg_ri_norms_naif,columns=attributes)

In [17]:
compareMethods(neg_variation_gradient,neg_variation_naif,attributes)

(7594, 1011)

### Positives

In [18]:
pos_ri,pos_ri_norms = computeRi(hybrid_model.classifier,attributes,positives,attribute_len,0)

 40%|████      | 26/65 [00:03<00:08,  4.71it/s]

can't converge in 100 iterations


 52%|█████▏    | 34/65 [00:05<00:06,  4.68it/s]

can't converge in 100 iterations


 54%|█████▍    | 35/65 [00:05<00:09,  3.26it/s]

...

can't converge in 100 iterations


 88%|████████▊ | 112/128 [00:20<00:04,  3.83it/s]

can't converge in 100 iterations


100%|██████████| 128/128 [00:22<00:00,  5.60it/s]

can't converge in 100 iterations


In [19]:
pos_ri_naif,pos_ri_norms_naif = computeRiNaif(positives,negatives,0,hybrid_model.classifier,attributes,attribute_len)

100%|██████████| 128/128 [02:04<00:00,  1.03it/s]


In [20]:
pos_perturbations_gradient = pd.DataFrame(data= pos_ri_norms,columns=attributes)
pos_perturbations_naif = pd.DataFrame(data= pos_ri_norms_naif,columns=attributes)

In [21]:
compareMethods(pos_perturbations_gradient,pos_perturbations_naif,attributes)

(570, 8)

In [27]:
!jt -t monokai